In [48]:
import ast
import requests
import time

api_key = "RGAPI-415a160c-532f-41e2-9a53-78299b01cb10"  # Thay thế bằng API key của bạn
cnt = 1

# Đọc tất cả các PUUID từ PlayerPUUID.txt
PlayerPUUID = 'ChallengerPlayerPUUID.txt'
PlayerMatchID = 'ChallengerPlayerMatchID.txt'
result = 'unprocessed_challenger.txt'
with open(PlayerPUUID, 'r') as puuid_file:
    target_puuids = [line.strip() for line in puuid_file]

# Đọc tất cả các mảng match IDs từ PlayerMatchID.txt
with open(PlayerMatchID, 'r') as match_file:
    match_id_arrays = [ast.literal_eval(line.strip()) for line in match_file]

requests_count = 0  # Đếm số lượng yêu cầu
start_time = time.time()  # Thời gian bắt đầu

# Lặp qua mỗi target_puuid cùng với mảng match_id tương ứng
for target_puuid, match_ids in zip(target_puuids, match_id_arrays):
    for match_id in match_ids:
        # Kiểm tra và giới hạn số lượng yêu cầu mỗi giây và mỗi 2 phút
        current_time = time.time()
        
        # Nếu có 20 requests trong 1 giây, chờ đến khi đủ 1 giây
        if requests_count >= 20 and current_time - start_time < 1:
            time.sleep(1 - (current_time - start_time))
            start_time = time.time()
            requests_count = 0  # Reset đếm số yêu cầu

        # Nếu có 100 requests trong 2 phút, chờ thêm thời gian
        if requests_count >= 100 and current_time - start_time < 120:
            time.sleep(121 - (current_time - start_time))
            start_time = time.time()
            requests_count = 0  # Reset đếm số yêu cầu

        url3 = f'https://sea.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
        
        retry_attempts = 5  # Số lần thử lại khi gặp lỗi
        while retry_attempts > 0:
            response3 = requests.get(url3)
            requests_count += 1  # Tăng đếm số yêu cầu

            if response3.status_code == 200:
                match_info = response3.json()
                participants = [p['puuid'] for p in match_info['info']['participants']]

                # Kiểm tra target_puuid trong danh sách participants
                if target_puuid in participants:
                    index_of_target = participants.index(target_puuid)
                    participant = match_info['info']['participants'][index_of_target]
                    kills = participant['kills']
                    deaths = participant['deaths']
                    assists = participant['assists']
                    kda = kills + assists if deaths == 0 else (kills + assists) / deaths
                    total_gold = participant['goldEarned']
                    total_minion = participant['totalMinionsKilled']
                    
                    # Ghi kết quả vào file
                    with open(result, 'a+') as result_file:
                        result_file.write(f"Kills: {kills} Deaths: {deaths} Assists: {assists} KDA: {kda:.2f} total_gold: {total_gold} total_minion: {total_minion}\n")
                    
                    print(f"Đã viết trận {cnt}  cho {target_puuid} - ")
                    cnt += 1
                else:
                    print(f"PUUID {target_puuid} không tìm thấy trong trận đấu {match_id}")
                    cnt += 1
                break  # Nếu thành công, thoát khỏi vòng lặp retry
            else:
                retry_attempts -= 1
                wait_time = 10 * (5 - retry_attempts)  # Tăng thời gian chờ sau mỗi lần thất bại
                print(f"Failed to retrieve: {match_id}. Status Code: {response3.status_code}. Thử lại sau {wait_time} giây...")
                time.sleep(wait_time)  # Chờ trước khi thử lại
print ('Đã hoàn thành!')

Đã viết trận 1  cho LUaGeYgfk9GEHLtELtRlA3HtdJo8ievYgD5ZiB-_trDEaEXnhgh6-jEYjLwFnJc-lXc8LgPicCZm_A - 
Đã viết trận 2  cho LUaGeYgfk9GEHLtELtRlA3HtdJo8ievYgD5ZiB-_trDEaEXnhgh6-jEYjLwFnJc-lXc8LgPicCZm_A - 
Đã viết trận 3  cho LUaGeYgfk9GEHLtELtRlA3HtdJo8ievYgD5ZiB-_trDEaEXnhgh6-jEYjLwFnJc-lXc8LgPicCZm_A - 
Đã viết trận 4  cho LUaGeYgfk9GEHLtELtRlA3HtdJo8ievYgD5ZiB-_trDEaEXnhgh6-jEYjLwFnJc-lXc8LgPicCZm_A - 
Đã viết trận 5  cho LUaGeYgfk9GEHLtELtRlA3HtdJo8ievYgD5ZiB-_trDEaEXnhgh6-jEYjLwFnJc-lXc8LgPicCZm_A - 
Đã viết trận 6  cho LUaGeYgfk9GEHLtELtRlA3HtdJo8ievYgD5ZiB-_trDEaEXnhgh6-jEYjLwFnJc-lXc8LgPicCZm_A - 
Đã viết trận 7  cho LUaGeYgfk9GEHLtELtRlA3HtdJo8ievYgD5ZiB-_trDEaEXnhgh6-jEYjLwFnJc-lXc8LgPicCZm_A - 
Đã viết trận 8  cho LUaGeYgfk9GEHLtELtRlA3HtdJo8ievYgD5ZiB-_trDEaEXnhgh6-jEYjLwFnJc-lXc8LgPicCZm_A - 
Đã viết trận 9  cho LUaGeYgfk9GEHLtELtRlA3HtdJo8ievYgD5ZiB-_trDEaEXnhgh6-jEYjLwFnJc-lXc8LgPicCZm_A - 
Đã viết trận 10  cho LUaGeYgfk9GEHLtELtRlA3HtdJo8ievYgD5ZiB-_trDEaEXnhgh6-jEYjLwFn

In [50]:
# Đọc file PlayerMatchID.txt để xác định số phần tử trong mỗi mảng
match_counts = []
with open(PlayerMatchID, 'r') as match_file:
    for line in match_file:
        # Chuyển chuỗi dạng mảng thành một mảng thực sự
        array = ast.literal_eval(line.strip())
        match_counts.append(len(array))  # Lưu số phần tử trong mỗi mảng

input_file = result
output_file = 'Challenger_final.txt'

with open(input_file, 'r') as file, open(output_file, 'a+') as output:
    # Ghi header
    output.write("Avg_Kills,Avg_Deaths,Avg_Assists,Avg_KDA,Avg_total_gold,Avg_total_minion\n")

    kills, deaths, assists, kda, total_gold, total_minion = 0, 0, 0, 0, 0, 0
    count = 0
    group_count = 0

    for line in file:
        # Trích xuất giá trị từ dòng
        parts = line.strip().split()
        kills += int(parts[1])
        deaths += int(parts[3])
        assists += int(parts[5])
        kda += float(parts[7])
        total_gold += int(parts[9])
        total_minion += int(parts[11])
        count += 1
        
        # Kiểm tra nếu đã đạt đủ số phần tử trong mảng tương ứng
        if count == match_counts[group_count]:  
            avg_kills = kills / match_counts[group_count]
            avg_deaths = deaths / match_counts[group_count]
            avg_assists = assists / match_counts[group_count]
            avg_kda = kda / match_counts[group_count]
            avg_total_gold = total_gold / match_counts[group_count]
            avg_total_minion = total_minion / match_counts[group_count]
            
            # Ghi các giá trị trung bình vào file output
            output.write(f"{avg_kills:.2f},{avg_deaths:.2f},{avg_assists:.2f},{avg_kda:.2f},{avg_total_gold:.2f},{avg_total_minion:.2f}\n")
            
            # Reset biến đếm và tổng
            kills, deaths, assists, kda, total_gold, total_minion = 0, 0, 0, 0, 0, 0
            count = 0
            group_count += 1

    print(f"Đã hoàn thành tính số liệu trung bình của {group_count} người chơi {output_file.split('_')[0]} rồi")


Đã hoàn thành tính số liệu trung bình của 98 người chơi Challenger rồi
